# Setup

- Make sure to have a clock visible
- Check network connectivity
- Displays mirrored
- Slides up
- Empty notebook
  - Ideally using 3.7-pre because it has better error messages: `python-37/bin/jupyter notebook`
  - Full screened (F11)
  - Hide header and toolbar
  - Turn on line numbers
- This loaded in a tab for a cheat sheet I guess
- No other windows on the same desktop

# Demonstrating getaddrinfo

Demo with `lbl.gov`, `facebook.com`, `debian.org`

In [2]:
import socket
socket.getaddrinfo("lbl.gov", "http")

[(<AddressFamily.AF_INET: 2>,
  <SocketKind.SOCK_STREAM: 1>,
  6,
  '',
  ('128.3.41.146', 80))]

# `async/await`

In [1]:
import trio

# Also demonstrate: await outside of an async function

def double(x):
    return 2 * x

async def double_sleep(x):
    duration = double(x)
    print("Going to sleep")
    await trio.sleep(duration)
    print("Woke up again!")
    return duration
    
async def quadruple_sleep(x):
    duration = await double_sleep(double(x))
    print("duration was", duration)

trio.run(quadruple_sleep, 1)

Going to sleep
Woke up again!
duration was 4


# Cancel scopes

In [29]:
# Timeouts

async def main():
    with trio.move_on_after(2):
        print("Going to sleep")
        await trio.sleep(999)
        print("Sleep finished")
    print("That's all folks")
        
trio.run(main)

Going to sleep
That's all folks


In [4]:
# Explicit cancellation

async def main():
    with trio.move_on_after(2) as cancel_scope:
        print("Cancel scope is", cancel_scope)
        cancel_scope.cancel()
        await quadruple_sleep(100)
    print("That's all folks")
        
trio.run(main)

Cancel scope is <cancel scope object at 0x7fa748013588>
Going to sleep
That's all folks


# Nurseries

In [36]:
async def task1():
    print("Task 1 started!")
    await trio.sleep(1)
    print("Task 1 finished!")
    
async def task2():
    print("Task 2 started")
    await trio.sleep(1)
    print("Task 2 finished")
    
async def main():
    print("main started")
    async with trio.open_nursery() as nursery:
        nursery.start_soon(task1)
        nursery.start_soon(task2)
    print("main finished")
    
trio.run(main)

main started
Task 2 started
Task 1 started!
Task 1 finished!
Task 2 finished
main finished


In [6]:
async def task1():
    print("Task 1 started!")
    await rtio.sleep(1)
    print("Task 1 finished!")
    
async def task2():
    print("Task 2 started")
    await trio.sleep(1)
    print("Task 2 finished")
    
async def main():
    print("main started")
    async with trio.open_nursery() as nursery:
        print("nursery cancel scope is", nursery.cancel_scope)        
        nursery.start_soon(task1)
        nursery.start_soon(task2)
    print("main finished")
    
trio.run(main)

main started
nursery cancel scope is <cancel scope object at 0x7fa748013cf8>
Task 1 started!
Task 2 started


NameError: name 'rtio' is not defined

In [ ]:
async def proxy_one_way(source, sink):
    while True:
        data = await source.read_some(MAX_SIZE)
        if not data:
            await sink.send_eof()
            return
        await sink.send_all(data)
        
async def proxy_two_way(a, b):
    async with trio.open_nursery() as nursery:
        nursery.start_soon(proxy_one_way, a, b)
        nursery.start_soon(proxy_one_way, b, a)

async def main():
    with move_on_after(10):
        await proxy_two_way(a, b)


# Basic inter-task communication

In [37]:
async def task1(event):
    print("task 1: waiting for event")
    await event.wait()
    print("task 1: the event happened!")
    
async def task2(event):
    print("task 2: sleeping")
    await trio.sleep(5)
    print("task 2: setting event")
    event.set()
    print("task 2: finished")
    
async def main():
    event = trio.Event()
    async with trio.open_nursery() as nursery:
        nursery.start_soon(task1, event)
        nursery.start_soon(task2, event)
        
trio.run(main)

task 1: waiting for event
task 2: sleeping
task 2: setting event
task 2: finished
task 1: the event happened!


# Happy eyeballs

In [27]:
TIMEOUT = 0.300

async def open_tcp_connection(host, port):
    targets = await trio.socket.getaddrinfo(host, port, type=trio.socket.SOCK_STREAM)
    winning_socket = None
    failure_events = [trio.Event() for _ in targets]
    
    async def attempt_one_connect(which, nursery):
        with trio.move_on_after(TIMEOUT):
            if which - 1 >= 0:
                await failure_events[which - 1].wait()
        
        if which + 1 < len(targets):
            nursery.start_soon(attempt_one_connect, which + 1, nursery)
        
        *socket_config, _, address = targets[which]
        try:
            socket = trio.socket.socket(*socket_config)
            await socket.connect(address)
        except OSError:
            failure_events[which].set()
        else:
            nonlocal winning_socket
            winning_socket = socket
            nursery.cancel_scope.cancel()
        
    async with trio.open_nursery() as nursery:
        nursery.start_soon(attempt_one_connect, 0, nursery)
        
    if winning_socket is None:
        raise OSError("all attempts failed")
    else:
        return winning_socket
    
async def main():
    #socket = await open_tcp_connection("www.debian.org", "https")
    #print(socket)
    #stream = trio.ssl.SSLStream(trio.SocketStream(socket), trio.ssl.create_default_context(), server_hostname="www.debian.org")
    stream = await trio.open_ssl_over_tcp_stream("www.debian.org", 443)
    await stream.send_all(b"GET / HTTP/1.1\r\nHost: www.debian.org\r\n\r\n")
    print((await stream.receive_some(10000)).decode("utf-8"))
    print((await stream.receive_some(10000)).decode("utf-8"))

trio.run(main)

HTTP/1.1 200 OK
Date: Fri, 01 Dec 2017 00:25:10 GMT
Server: Apache
Content-Location: index.en.html
Vary: negotiate,accept-language,Accept-Encoding
TCN: choice
X-Content-Type-Options: nosniff
X-Frame-Options: sameorigin
Referrer-Policy: no-referrer
X-Xss-Protection: 1
Strict-Transport-Security: max-age=15552000
Last-Modified: Thu, 30 Nov 2017 19:26:39 GMT
ETag: "3ce6-55f383b2b6373"
Accept-Ranges: bytes
Content-Length: 15590
Cache-Control: max-age=86400
Expires: Sat, 02 Dec 2017 00:25:10 GMT
X-Clacks-Overhead: GNU Terry Pratchett
Content-Type: text/html
Content-Language: en


<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01//EN" "http://www.w3.org/TR/html4/strict.dtd">
<html lang="en">
<head>
  <meta http-equiv="Content-Type" content="text/html; charset=utf-8">
  <title>Debian -- The Universal Operating System </title>
  <link rel="author" href="mailto:webmaster@debian.org">
  <meta name="Description" content="Debian is an operating system and a distribution of Free Software. It is maintaine